# Snee's Contingency Table

The following contingency table was given in Snee, "Graphical display of two-way contingency tables." Amer. Statist. 38 9᎐12. (1974). It satisfies a standard criterion of at least 5 counts per cell for application of [Pearson's chi-squared test](https://en.wikipedia.org/wiki/Pearson's_chi-squared_test) for independence.  

Eyes\Hair | Black | Brunette | Red | Blonde
--------|--------|---------|-------|-------
Brown | 68 | 119 | 26 | 7
Blue | 20 | 84 | 17 | 94
Hazel | 15 | 54 | 14 | 10
Green | 5 | 29 | 14 | 16

The value of the test statistic for this table is 138.3, which in the case of independence (and validity of the test) would have likelihood less than $10^{-16}$. The test thus indicates a strong association between eye color and hair color.

For instance, the table shown below, having the same row and column sums as the one above, has a test stastistic of 0.088, which is well within the range expected in the case of independence.

lc\UC | A | B | C | D
----|----|----|----|---
a | 40 | 106 | 26 | 48
b | 39 | 104 | 26 | 46
c | 17 | 45 | 11  | 20
d | 12 | 31 | 8 | 13

This table, however, is special. It was designed to have entries close to their expected values given independence and the specified row and column sums. Would "random" tables with these row and column sums have similar test statistic values? To find out, MCMC can be used to generate such tables in a uniformly random manner.

The example below follows "Hit and Run as a Unifying Device." by P. Diaconis & H. Andersen, (2007), *Journal de la Société Française de Statistique*, 148(4):5-28, [[PDF](http://statweb.stanford.edu/~cgates/PERSI/papers/hitandrun062207.pdf)] and its references.

The example relies on packages `Mamba,` `Plots,` and `PyPlot` in addition to `HitAndRun`.

In [1]:
include("../src/HitAndRun.jl") # use this line only if running from source, as here
using HitAndRun # if HitAndRun is installed as a package, only this line is needed
snee

4x4 Array{Int64,2}:
 68  119  26   7
 20   84  17  94
 15   54  14  10
  5   29  14  16

We use a hit and run algorithm to generate a series of tables approximately distributed as uniformly random samples from 4x4 contingency tables with the same row and column sums as `snee`. The algorithm is expected to converge rapidly. Tentatively, we skip the first 1000 to allow for convergence. Since sequential tables are very similar, we save only 1 table out of every 50 generated in sequence.

In [8]:
srand(1234)
tables = mcmc_tables(snee, 1000, 50, 100000; uniform=true);
summary(tables)

"4x4x1980 Array{Int64,3}"

We show that generated tables have the same row and column sums as `snee`. First columns:

In [19]:
rtab = tables[:,:,rand(1:size(tables,3))]; # randomly selected table from among those generated
hcat(sum(snee,2), sum(rtab,2))

4x2 Array{Int64,2}:
 220  220
 215  215
  93   93
  64   64

then rows:

In [20]:
vcat(sum(snee,1), sum(rtab,1))

2x4 Array{Int64,2}:
 108  286  71  127
 108  286  71  127

Rather than test `tables` itself for convergence, which can be done, we instead compute the associated sequence of chi-squared statistics and test this for convergence. To do so, we convert the chi-squared sequence to an object of type `Mamba.Chains` to which we will apply Mamba's [Geweke diagnostic](http://mambajl.readthedocs.io/en/latest/mcmc/chains.html#geweke-diagnostic).

In [9]:
chisqs = chisq_tables(tables);
chain = tables2chains(chisqs, 1000, 50)

Object of type "Mamba.Chains"

Iterations = 1050:100000
Thinning interval = 50
Chains = 1
Samples per chain = 1980

1980x1x1 Array{Float64,3}:
[:, :, 1] =
 181.429 
 208.978 
 204.714 
 312.114 
 150.022 
  56.5959
 191.868 
 196.058 
 361.363 
  68.6817
 177.521 
 118.58  
 360.576 
   ⋮     
  52.9965
 282.025 
 309.893 
 210.907 
 161.328 
 218.352 
 414.949 
 291.245 
 262.178 
 161.864 
 344.216 
 152.776 

Note that the representative chi-squared values are closer to that of `snee`, 138.3, rather than that of the special table, 0.088. Provided the chain has converged, this would suggest the Pearson test is misleading in this case. We test convergence as follows.

In [10]:
using Mamba
gewekediag(chain)

  Z-score p-value
T  -0.843   0.399



The p-value of 0.399 is consistent with convergence, the Geweke diagnostic's null hypothesis. Other tests give similar results. (The [Gelman, Rubin, and Brooks diagnostic](http://mambajl.readthedocs.io/en/latest/mcmc/chains.html#gelman-rubin-and-brooks-diagnostics) requires at least two chains. Appropriate input can be formed from single `Mamba.Chains` objects using [vcat](http://mambajl.readthedocs.io/en/latest/mcmc/chains.html#vcat).)

Finally, we plot a histogram of the sampled chi-squared scores, indicating `snee`'s score in red. It can be seen that `snee`'s score is not atypical.

In [11]:
using Plots
pyplot()
histogram(chisqs)
vline!([chisq_tables(snee)], color=:red, lw=5)

[Plots.jl] Initializing backend: pyplot

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 <path d="
M31.7812 66.4062
Q24.1719 66.4062 20.3281 58.9062
Q16.5 51.4219 16.5 36.375
Q16.5 21.3906 20.3281 13.8906
Q24.1719 6.39062 31.7812 6.39062
Q39.4531 6.39062 43.2812 13.8906
Q47.125 21.3906 47.125 36.375
Q47.125 51.4219 43.2812 58.9062
Q39.4531 66.4062 31.7812 66.4062
M31.7812 74.2188
Q44.0469 74.2188 50.5156 64.5156
Q56.9844 54.8281 56.9844 36.375
Q56.9844 17.9688 50.5156 8.26562
Q44.0469 -1.42188 31.7812 -1.42188
Q19.5312 -1.42188 13.0625 8.26562
Q6.59375 17.9688 6.59375 36.375
Q6.59375 54.8281 13.0625 64.5156
Q19.5312 74.2188 31.7812 74.2188" id="DejaVuSans-30"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 100 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 200 -->
 
 <path d="
M19.1875 8.29688
L53.6094 8.29688
L53.6094 0
L7.32812 0
L7.32812 8.29688
Q12.9375 14.1094 22.625 23.8906
Q32.3281 33.6875 34.8125 36.5312
Q39.5469 41.8438 41.4219 45.5312
Q43.3125 49.2188 43.3125 52.7812
Q43.3125 58.5938 39.2344 62.25
Q35.1562 65.9219 28.6094 65.9219
Q23.9688 65.9219 18.8125 64.3125
Q13.6719 62.7031 7.8125 59.4219
L7.8125 69.3906
Q13.7656 71.7812 18.9375 73
Q24.125 74.2188 28.4219 74.2188
Q39.75 74.2188 46.4844 68.5469
Q53.2188 62.8906 53.2188 53.4219
Q53.2188 48.9219 51.5312 44.8906
Q49.8594 40.875 45.4062 35.4062
Q44.1875 33.9844 37.6406 27.2188
Q31.1094 20.4531 19.1875 8.29688" id="DejaVuSans-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 300 -->
 
 <path d="
M40.5781 39.3125
Q47.6562 37.7969 51.625 33
Q55.6094 28.2188 55.6094 21.1875
Q55.6094 10.4062 48.1875 4.48438
Q40.7656 -1.42188 27.0938 -1.42188
Q22.5156 -1.42188 17.6562 -0.515625
Q12.7969 0.390625 7.625 2.20312
L7.625 11.7188
Q11.7188 9.32812 16.5938 8.10938
Q21.4844 6.89062 26.8125 6.89062
Q36.0781 6.89062 40.9375 10.5469
Q45.7969 14.2031 45.7969 21.1875
Q45.7969 27.6406 41.2812 31.2656
Q36.7656 34.9062 28.7188 34.9062
L20.2188 34.9062
L20.2188 43.0156
L29.1094 43.0156
Q36.375 43.0156 40.2344 45.9219
Q44.0938 48.8281 44.0938 54.2969
Q44.0938 59.9062 40.1094 62.9062
Q36.1406 65.9219 28.7188 65.9219
Q24.6562 65.9219 20.0156 65.0312
Q15.375 64.1562 9.8125 62.3125
L9.8125 71.0938
Q15.4375 72.6562 20.3438 73.4375
Q25.25 74.2188 29.5938 74.2188
Q40.8281 74.2188 47.3594 69.1094
Q53.9062 64.0156 53.9062 55.3281
Q53.9062 49.2656 50.4375 45.0938
Q46.9688 40.9219 40.5781 39.3125" id="DejaVuSans-33"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 400 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 500 -->
 
 <path d="
M10.7969 72.9062
L49.5156 72.9062
L49.5156 64.5938
L19.8281 64.5938
L19.8281 46.7344
Q21.9688 47.4688 24.1094 47.8281
Q26.2656 48.1875 28.4219 48.1875
Q40.625 48.1875 47.75 41.5
Q54.8906 34.8125 54.8906 23.3906
Q54.8906 11.625 47.5625 5.09375
Q40.2344 -1.42188 26.9062 -1.42188
Q22.3125 -1.42188 17.5469 -0.640625
Q12.7969 0.140625 7.71875 1.70312
L7.71875 11.625
Q12.1094 9.23438 16.7969 8.0625
Q21.4844 6.89062 26.7031 6.89062
Q35.1562 6.89062 40.0781 11.3281
Q45.0156 15.7656 45.0156 23.3906
Q45.0156 31 40.0781 35.4375
Q35.1562 39.8906 26.7031 39.8906
Q22.75 39.8906 18.8125 39.0156
Q14.8906 38.1406 10.7969 36.2812
z
" id="DejaVuSans-35"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 600 -->
 
 <path d="
M33.0156 40.375
Q26.375 40.375 22.4844 35.8281
Q18.6094 31.2969 18.6094 23.3906
Q18.6094 15.5312 22.4844 10.9531
Q26.375 6.39062 33.0156 6.39062
Q39.6562 6.39062 43.5312 10.9531
Q47.4062 15.5312 47.4062 23.3906
Q47.4062 31.2969 43.5312 35.8281
Q39.6562 40.375 33.0156 40.375
M52.5938 71.2969
L52.5938 62.3125
Q48.875 64.0625 45.0938 64.9844
Q41.3125 65.9219 37.5938 65.9219
Q27.8281 65.9219 22.6719 59.3281
Q17.5312 52.7344 16.7969 39.4062
Q19.6719 43.6562 24.0156 45.9219
Q28.375 48.1875 33.5938 48.1875
Q44.5781 48.1875 50.9531 